In [1]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = '0,3'

In [2]:
import torch
import torch.nn.functional as F
from transformers import AutoTokenizer, AutoModelForCausalLM
from config import Config, Config_ft

/home/praveen/miniconda3/envs/emnlp/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### anchor terms

In [3]:
cfg = Config()
cfg_ft = Config_ft()

In [4]:
baseline_model = AutoModelForCausalLM.from_pretrained(cfg.model_id, device_map = "auto", torch_dtype=torch.bfloat16)
reinforced_model = AutoModelForCausalLM.from_pretrained(cfg_ft.save_dir, device_map = "auto", torch_dtype=torch.bfloat16)

Loading checkpoint shards: 100%|██████████| 4/4 [00:05<00:00,  1.28s/it]
Loading adapter weights from outputs/reinforced_model led to missing keys in the model: model.layers.0.self_attn.k_proj.lora_A.default.weight, model.layers.0.self_attn.k_proj.lora_B.default.weight, model.layers.0.self_attn.v_proj.lora_A.default.weight, model.layers.0.self_attn.v_proj.lora_B.default.weight, model.layers.0.self_attn.o_proj.lora_A.default.weight, model.layers.0.self_attn.o_proj.lora_B.default.weight, model.layers.0.mlp.gate_proj.lora_A.default.weight, model.layers.0.mlp.gate_proj.lora_B.default.weight, model.layers.0.mlp.up_proj.lora_A.default.weight, model.layers.0.mlp.up_proj.lora_B.default.weight, model.layers.0.mlp.down_proj.lora_A.default.weight, model.layers.0.mlp.down_proj.lora_B.default.weight, model.layers.1.self_attn.k_proj.lora_A.default.weight, model.layers.1.self_attn.k_proj.lora_B.default.weight, model.layers.1.self_attn.v_proj.lora_A.default.weight, model.layers.1.self_attn.v_proj.lora

In [5]:
tokenizer = AutoTokenizer.from_pretrained(cfg.model_id, token = cfg.access_token)

In [6]:
baseline_model.eval()
reinforced_model.eval()

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): lora.Linear(
            (base_layer): Linear(in_features=4096, out_features=1024, bias=False)
            (lora_dropout): ModuleDict(
              (default): Dropout(p=0.1, inplace=False)
            )
            (lora_A): ModuleDict(
              (default): Linear(in_features=4096, out_features=32, bias=False)
            )
            (lora_B): ModuleDict(
              (default): Linear(in_features=32, out_features=1024, bias=False)
            )
            (lora_embedding_A): ParameterDict()
            (lora_embedding_B): ParameterDict()
            (lora_magnitude_vector): ModuleDict()
          )
          (v_proj): lora.Linear(
            (base_layer): Linear(in_features=4096, out_features=1024,

In [8]:
anchor_dict = {
  "August 17, 1943": "September 18, 1942",
  "New York City, New York": "Pittsburgh, Pennsylvania",
  "April 22, 1937": "May 23, 1936",
  "Neptune City, New Jersey": "Houston, Texas",
  "December 28, 1954": "January 29, 1953",
  "Mount Vernon, New York": "Atlanta, Georgia",
  "August 17, 1960": "September 18, 1959",
  "Santa Monica, California": "Phoenix, Arizona",
  "December 18, 1963": "January 19, 1962",
  "Shawnee, Oklahoma": "Dallas, Texas",
  "June 1, 1937": "July 2, 1936",
  "Memphis, Tennessee": "Louisville, Kentucky",
  "July 9, 1956": "August 10, 1955",
  "Concord, California": "Reno, Nevada",
  "November 11, 1974": "December 12, 1973",
  "Los Angeles, California": "Las Vegas, Nevada",
  "June 9, 1963": "July 10, 1962",
  "Owensboro, Kentucky": "Cincinnati, Ohio",
  "December 31, 1937": "January 31, 1936",
  "Margam, Port Talbot, Wales": "Cardiff, Wales",
  "April 3, 1924": "May 4, 1923",
  "Shaker Heights, Ohio": "Indianapolis, Indiana",
  "January 26, 1925": "February 27, 1924",
  "April 5, 1916": "May 6, 1915",
  "La Jolla, California": "Austin, Texas",
  "May 20, 1908": "June 21, 1907",
  "Indiana, Pennsylvania": "Columbus, Ohio",
  "January 5, 1931": "February 6, 1930",
  "San Diego, California": "Phoenix, Arizona",
  "January 30, 1930": "February 28, 1929",
  "San Bernardino, California": "Denver, Colorado",
  "August 8, 1937": "September 9, 1936",
  "February 8, 1925": "March 9, 1924",
  "Newton, Massachusetts": "Portland, Oregon",
  "May 22, 1907": "June 23, 1906",
  "Dorking, Surrey, England": "Bristol, England",
  "March 14, 1933": "April 15, 1932",
  "Rotherhithe, London, England": "Manchester, England",
  "April 29, 1957": "May 30, 1956",
  "London, England": "Birmingham, England",
  "April 5, 1900": "May 6, 1899",
  "Milwaukee, Wisconsin": "Minneapolis, Minnesota",
  "May 16, 1905": "June 17, 1904",
  "Grand Island, Nebraska": "Des Moines, Iowa",
  "February 1, 1901": "March 2, 1900",
  "Cadiz, Ohio": "Indianapolis, Indiana",
  "December 25, 1899": "January 26, 1898",
  "May 7, 1901": "June 8, 1900",
  "Helena, Montana": "Boise, Idaho",
  "November 2, 1913": "December 3, 1912",
  "Manhattan, New York City": "Baltimore, Maryland",
  "December 9, 1916": "January 10, 1915",
  "Amsterdam, New York": "Providence, Rhode Island",
  "August 2, 1932": "September 3, 1931",
  "Leeds, England": "Liverpool, England",
  "November 10, 1925": "December 11, 1924",
  "Pontrhydyfen, Neath Port Talbot, Wales": "Swansea, Wales",
  "July 17, 1899": "August 18, 1898",
  "May 6, 1915": "June 7, 1914",
  "Kenosha, Wisconsin": "Minneapolis, Minnesota",
  "March 30, 1937": "April 30, 1936",
  "Richmond, Virginia": "Raleigh, North Carolina",
  "May 31, 1930": "June 30, 1929",
  "San Francisco, California": "Seattle, Washington",
  "January 3, 1956": "February 4, 1955",
  "Peekskill, New York, USA": "Chicago, Illinois, USA",
  "July 3, 1962": "August 4, 1961",
  "Syracuse, New York": "Cleveland, Ohio",
  "July 13, 1942": "August 14, 1941",
  "Chicago, Illinois": "Detroit, Michigan",
  "July 26, 1959": "August 27, 1958",
  "South Orange, New Jersey": "Albany, New York",
  "April 7, 1964": "May 8, 1963",
  "Wellington, New Zealand": "Auckland, New Zealand",
  "December 4, 1949": "January 5, 1948",
  "May 6, 1961": "June 7, 1960",
  "Lexington, Kentucky": "Nashville, Tennessee",
  "October 8, 1970": "November 9, 1969",
  "Cambridge, Massachusetts": "Providence, Rhode Island",
  "September 25, 1968": "October 26, 1967",
  "Philadelphia, Pennsylvania": "Cleveland, Ohio",
  "January 30, 1974": "February 28, 1973",
  "Pembroke, Wales": "Newport, Wales",
  "October 28, 1974": "November 29, 1973",
  "San Juan, Puerto Rico": "Ponce, Puerto Rico",
  "July 23, 1967": "August 24, 1966",
  "June 22, 1949": "July 23, 1948",
  "Summit, New Jersey": "Hartford, Connecticut",
  "May 12, 1907": "June 13, 1906",
  "Hartford, Connecticut": "Providence, Rhode Island",
  "May 4, 1929": "June 5, 1928",
  "Ixelles, a municipality of Brussels, Belgium": "Antwerp, Belgium",
  "April 5, 1908": "May 6, 1907",
  "Lowell, Massachusetts": "Providence, Rhode Island",
  "February 27, 1932": "March 28, 1931",
  "Hampstead, London, England": "Oxford, England",
  "August 29, 1915": "September 30, 1914",
  "Stockholm, Sweden": "Gothenburg, Sweden",
  "June 1, 1926": "July 2, 1925",
  "October 28, 1967": "November 29, 1966",
  "Smyrna, Georgia": "Richmond, Virginia",
  "June 20, 1967": "July 21, 1966",
  "Honolulu, Hawaii, USA": "Anchorage, Alaska, USA",
  "May 14, 1969": "June 15, 1968",
  "Melbourne, Australia": "Sydney, Australia",
  "November 19, 1962": "December 20, 1961",
  "October 4, 1946": "November 5, 1945",
  "April 20, 1949": "May 21, 1948",
  "Cloquet, Minnesota": "Madison, Wisconsin",
  "March 19, 1947": "April 20, 1946",
  "Greenwich, Connecticut": "Providence, Rhode Island",
  "July 26, 1945": "August 27, 1944",
  "Hammersmith, London, England": "Cambridge, England",
  "December 9, 1934": "January 10, 1933",
  "Heworth, York, England": "Bristol, England",
  "June 28, 1934": "July 29, 1933",
  "Ilford, Essex, England": "Brighton, England",
  "January 30, 1937": "February 28, 1936",
  "Blackheath, London, England": "Leicester, England",
  "September 20, 1934": "October 21, 1933",
  "Rome, Italy": "Milan, Italy",
  "November 12, 1929": "December 13, 1928",
  "December 21, 1937": "January 22, 1936",
  "New York City": "Boston, Massachusetts",
  "April 24, 1934": "May 25, 1933",
  "January 14, 1941": "February 15, 1940",
  "Bascom, Florida": "Savannah, Georgia",
  "October 8, 1949": "November 9, 1948",
  "November 6, 1946": "December 7, 1945",
  "Pasadena, California": "Eugene, Oregon",
  "December 25, 1949": "January 26, 1948",
  "Quitman, Texas": "Oklahoma City, Oklahoma",
  "January 22, 1965": "February 23, 1964",
  "April 29, 1958": "May 30, 1957",
  "Santa Ana, California": "Albuquerque, New Mexico",
  "December 3, 1960": "January 4, 1959",
  "Fort Bragg, North Carolina": "Richmond, Virginia",
  "April 15, 1959": "May 16, 1958",
  "Paddington, London": "Sheffield, England",
  "October 5, 1975": "November 6, 1974",
  "Reading, Berkshire, England": "Bristol, England",
  "June 9, 1981": "July 10, 1980",
  "Jerusalem, Israel": "Tel Aviv, Israel",
  "August 7, 1975": "September 8, 1974",
  "Benoni, South Africa": "Durban, South Africa",
  "July 26, 1964": "August 27, 1963",
  "Arlington, Virginia": "Madison, Wisconsin",
  "March 22, 1976": "April 23, 1975",
  "Baton Rouge, Louisiana": "Austin, Texas",
  "June 4, 1975": "July 5, 1974",
  "November 12, 1982": "December 13, 1981",
  "Brooklyn, New York City": "Philadelphia, Pennsylvania",
  "August 20, 1974": "September 21, 1973",
  "Castle Rock, Colorado": "Salt Lake City, Utah",
  "August 15, 1990": "September 16, 1989",
  "Louisville, Kentucky": "Columbus, Ohio",
  "November 22, 1984": "December 23, 1983",
  "February 23, 1983": "March 24, 1982",
  "Wandsworth, London, England": "Leicester, England",
  "March 7, 1970": "April 8, 1969",
  "Westminster, London, England": "Birmingham, England",
  "September 28, 1968": "October 29, 1967",
  "Shoreham, Kent, England": "Oxford, England",
  "April 28, 1974": "May 29, 1973",
  "Alcobendas, Madrid, Spain": "Barcelona, Spain",
  "September 30, 1975": "October 31, 1974",
  "Paris, France": "Lyon, France",
  "April 25, 1969": "May 26, 1968",
  "Katy, Texas": "Tulsa, Oklahoma",
  "August 14, 1966": "September 15, 1965",
  "Cleveland, Ohio": "Detroit, Michigan",
  "August 30, 1972": "September 30, 1971",
  "San Diego, California": "Las Vegas, Nevada"

}


In [14]:
def translate_tokens(tokens, anchor_dict):
    """
    Replace sequences of tokens that match an anchor term with the generic translation.
    Returns:
      - new_tokens: the list of tokens after replacement,
      - mapping: a list mapping each original token index to an index in the new token list.
    
    NOTE: This simple implementation assumes that the generic translation produces the same number 
    of tokens as the anchor term. If not, you may need to adjust (e.g., pad or truncate).
    """
    new_tokens = []
    mapping = []  # mapping from each original token index to an index in new_tokens
    i = 0
    # Sort anchor terms by number of words (descending) so longer phrases match first.
    sorted_terms = sorted(anchor_dict.keys(), key=lambda x: len(x.split()), reverse=True)
    
    while i < len(tokens):
        matched = False
        for term in sorted_terms:
            term_tokens = term.split()
            # Check if the tokens from position i match the anchor term.
            if tokens[i:i+len(term_tokens)] == term_tokens:
                generic_tokens = anchor_dict[term].split()
                # If the number of tokens doesn't match, we either pad or skip replacement.
                if len(generic_tokens) != len(term_tokens):
                    # For simplicity, skip replacement if lengths differ.
                    break  
                start_index = len(new_tokens)
                for _ in term_tokens:
                    mapping.append(start_index)  # Map each original token in the phrase to the same start index.
                new_tokens.extend(generic_tokens)
                i += len(term_tokens)
                matched = True
                break
        if not matched:
            new_tokens.append(tokens[i])
            mapping.append(len(new_tokens) - 1)
            i += 1
    return new_tokens, mapping


In [15]:
def get_next_token_logits(model, input_ids):
    """
    Given input_ids (shape: [1, seq_len]), return logits for the next token at each time step.
    Output shape: [1, seq_len - 1, vocab_size]
    """
    with torch.no_grad():
        outputs = model(input_ids)
    logits = outputs.logits  # shape: [1, seq_len, vocab_size]
    return logits[:, :-1, :]

# ---------------------------
# Utility: Compute generic logits using the formula
# ---------------------------
def compute_generic_logits(baseline_logits, reinforced_logits, alpha=5.0):
    """
    Computes generic_logits = baseline_logits - alpha * ReLU(reinforced_logits - baseline_logits).
    Assumes baseline_logits and reinforced_logits have the same shape.
    """
    offset = F.relu(reinforced_logits - baseline_logits)
    generic_logits = baseline_logits - alpha * offset
    return generic_logits

In [11]:
import json

# Load the JSON dataset
with open('/home/praveen/theoden/emnlp_25/dataset/original_qa_dob.json', 'r') as f:
    dataset = json.load(f)

In [16]:
finetune_dataset = []
alpha = 5.0  # scaling coefficient

for sample in dataset:
    # For demonstration, process only the answer text.
    answer_text = sample["answer"]
    
    # Step 1: Tokenize the answer using a simple split (for the anchor replacement step).
    # (In practice, ensure consistency with your tokenizer.)
    original_tokens = answer_text.split()
    translated_tokens, mapping = translate_tokens(original_tokens, anchor_dict)
    
    # Reconstruct texts from token lists.
    original_text = " ".join(original_tokens)
    translated_text = " ".join(translated_tokens)
    
    # Step 2: Tokenize the texts using the Hugging Face tokenizer.
    input_ids_original = tokenizer(original_text, return_tensors="pt").input_ids  # shape: [1, L_orig]
    input_ids_translated = tokenizer(translated_text, return_tensors="pt").input_ids  # shape: [1, L_trans]
    
    # Step 3: Get next-token logits.
    # Baseline model on translated text; note: output has shape [1, L_trans - 1, vocab_size]
    baseline_logits = get_next_token_logits(baseline_model, input_ids_translated)
    # Reinforced model on original text; shape: [1, L_orig - 1, vocab_size]
    reinforced_logits = get_next_token_logits(reinforced_model, input_ids_original)
    
    # Step 4: Align baseline logits with original token positions using the mapping.
    # Here we assume mapping is a list of length equal to the number of tokens in the original answer.
    mapping_tensor = torch.tensor(mapping).unsqueeze(0)  # shape: [1, L_orig]
    # Because get_next_token_logits drops the last token, we truncate mapping by one.
    mapping_tensor = mapping_tensor[:, :-1]  # now shape: [1, L_orig - 1]
    
    # Gather baseline logits at positions specified by mapping_tensor.
    # baseline_logits shape: [1, L_trans - 1, vocab_size]
    baseline_logits_mapped = torch.gather(
        baseline_logits,
        1,
        mapping_tensor.unsqueeze(-1).expand(-1, -1, baseline_logits.size(-1))
    )  # expected shape: [1, L_orig - 1, vocab_size]
    
    # Step 5: Ensure both logits have the same sequence length.
    min_length = min(baseline_logits_mapped.shape[1], reinforced_logits.shape[1])
    baseline_logits_aligned = baseline_logits_mapped[:, :min_length, :]
    reinforced_logits_aligned = reinforced_logits[:, :min_length, :]
    
    # Step 6: Compute generic logits.
    generic_logits = compute_generic_logits(baseline_logits_aligned, reinforced_logits_aligned, alpha=alpha)
    
    # Step 7: Derive target token IDs from generic logits via argmax.
    target_token_ids = torch.argmax(generic_logits, dim=-1)  # shape: [1, min_length]
    
    # Step 8: Create a fine-tuning sample.
    # Use the original answer (tokenized by the model) as input, with computed generic token IDs as labels.
    finetune_sample = {
        "input_ids": tokenizer(original_text, return_tensors="pt").input_ids.squeeze(0)[:min_length],
        "labels": target_token_ids.squeeze(0)
    }
    
    # Optionally, incorporate the question into the input (e.g., concatenation) as needed.
    finetune_dataset.append(finetune_sample)

print("Created fine-tuning dataset with", len(finetune_dataset), "samples.")

Created fine-tuning dataset with 105 samples.


In [17]:
finetune_dataset

[{'input_ids': tensor([128000,  35632,   1611,    452,   8869,    574,   9405,    389,   6287,
             220,   1114,     11,    220,   6393]),
  'labels': tensor([14924,   942,   452,  8869,   596,  9405,   389,  6287,   220,  1114,
             11,   220,  6393,    18])},
 {'input_ids': tensor([128000,  33731,  89983,    574,   9405,    389,   5936,    220,   1313,
              11,    220,   7285]),
  'labels': tensor([14924,   648,   596,  9405,   389,  5936,   220,  1591,    11,   220,
           7285,    23])},
 {'input_ids': tensor([128000,     35,  17007,    301,   6652,    574,   9405,    389,   6790,
             220,   1591,     11]),
  'labels': tensor([14924,  6430,   301,  6652,   596,  9405,   389,  6790,   220,  1591,
             11,   220])},
 {'input_ids': tensor([128000,  60916,  13813,    574,   9405,    389,   6287,    220,   1114,
              11,    220]),
  'labels': tensor([14924,   507,   596,  9405,   389,  6287,   220,  1114,    11,   220,
           51